In [12]:
## Import Packages
import praw
import requests
import json
import pandas as pd
from datetime import datetime
from pprint import pprint
import time
pd.set_option("display.max_columns",None)

In [13]:
## Create Function for Reddit Scraping
def get_pushshift_data(data_type, **kwargs):
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url,params=payload)
    return request.json()

In [31]:
# Set variables for retrieval of data

dates = pd.date_range(datetime(2020,1,1),datetime(2021,2,1),freq = 'D')
epoch = [int(date.timestamp()) for date in dates]
sort_type = "created_utc"
size = 50
sort = "desc"

In [33]:
## All Posts (r/WSB q = tesla) 

data_type = "submission"
query = "tesla"
subreddit = "wallstreetbets"
WSBP = pd.DataFrame()

for i in range(len(epoch)-1):
    while True:
        try:
            a = epoch[i] + 1
            b = epoch[i+1]
            data = get_pushshift_data(data_type = data_type,
                                  subreddit = subreddit,
                                  q = query,
                                  after= a,
                                  before = b,
                                  size = size,
                                  sort_type = sort_type,
                                  sort= sort)
            temp_df = pd.DataFrame(data["data"])
            WSBP = pd.concat([temp_df,WSBP])
            i += 1
        except ValueError:
            continue
        break

In [34]:
# Clean Scraped Data from WSB Posts

WSBP["created_utc"] = WSBP["created_utc"].apply(lambda x: datetime.fromtimestamp(x)) #Used only for first-time runs
WSBP["Date"] = pd.to_datetime(WSBP["created_utc"],format=('%Y%m%d')).dt.date
variables = ["title","Date","created_utc","num_comments","permalink","subreddit"]
WSBPosts= WSBP[variables]
WSBPosts["Post/Comment"] = "P"
WSBPosts.head()

#WSBPosts.to_csv("WSBP.csv",header=True) ## Saving to csv format

C:\Users\herow\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,title,Date,created_utc,num_comments,permalink,subreddit,Post/Comment
0,Hi guys.. anyone long on ABML.. I am doing som...,2021-02-01,2021-02-01 07:51:58,0,/r/wallstreetbets/comments/l9php8/hi_guys_anyo...,wallstreetbets,P
1,Nancy Pelosi bought Apple &amp; Tesla Calls,2021-02-01,2021-02-01 07:51:51,0,/r/wallstreetbets/comments/l9phm7/nancy_pelosi...,wallstreetbets,P
2,Anyone who shorts Tesla and daddy Elon gets sq...,2021-02-01,2021-02-01 07:26:00,0,/r/wallstreetbets/comments/l9ozbr/anyone_who_s...,wallstreetbets,P
3,Tesla,2021-02-01,2021-02-01 07:23:36,0,/r/wallstreetbets/comments/l9oxkb/tesla/,wallstreetbets,P
4,Tesla competition?,2021-02-01,2021-02-01 07:03:12,0,/r/wallstreetbets/comments/l9oif8/tesla_compet...,wallstreetbets,P


In [35]:
## All Posts (r/teslainvestorsclub, no specific query)

data_type = "submission"
subreddit = "teslainvestorsclub"
TICP = pd.DataFrame()

for i in range(len(epoch)-1):
    while True:
        try:
            a = epoch[i] + 1
            b = epoch[i+1]
            data = get_pushshift_data(data_type = data_type,
                                  subreddit = subreddit,
                                  #q = query,
                                  after= a,
                                  before = b,
                                  size = size,
                                  sort_type = sort_type,
                                  sort= sort)
            temp_df = pd.DataFrame(data["data"])
            TICP = pd.concat([temp_df,TICP])
            i += 1
        except ValueError:
            continue
        break

In [48]:
TICP

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_richtext,link_flair_text_color,link_flair_type,locked,media,media_embed,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,retrieved_on,score,secure_media,secure_media_embed,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,edited,link_flair_template_id,link_flair_text,removed_by_category,crosspost_parent,crosspost_parent_list,banned_by,media_metadata,author_cakeday,distinguished,gilded,poll_data,steward_reports,Date
0,[],False,Fyx0z,#b8001f,ticowner,[],ba9649b8-3fa4-11ea-819b-0e7f2f6cbe83,Owner / Shareholder,light,text,t2_flfv5,False,False,[],False,False,2021-02-01 07:46:55,twitter.com,https://www.reddit.com/r/teslainvestorsclub/co...,{},l9pe4y,True,False,False,False,True,False,False,,[],dark,text,False,"{'oembed': {'author_name': 'Troy Teslike', 'au...","{'content': '&lt;blockquote class=""twitter-vid...",False,False,50,0,False,all_ads,/r/teslainvestorsclub/comments/l9pe4y/184k_1q2...,False,link,"{'enabled': False, 'images': [{'id': 'kwTf84Yd...",6.0,1612421014,108,"{'oembed': {'author_name': 'Troy Teslike', 'au...","{'content': '&lt;blockquote class=""twitter-vid...",,True,False,False,teslainvestorsclub,t5_n9evv,43885,public,confidence,https://b.thumbs.redditmedia.com/c9p4uY39xprso...,53.0,140.0,184k 1Q21 &amp; 871k 2021 Delivery Estimate - ...,0,[],0.99,https://twitter.com/troyteslike/status/1355987...,https://twitter.com/troyteslike/status/1355987...,all_ads,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-01
1,[],False,siennasolo,NaN,None,[],NaN,None,NaN,text,t2_9gr3pqlr,False,False,[],False,False,2021-02-01 07:25:43,self.teslainvestorsclub,https://www.reddit.com/r/teslainvestorsclub/co...,{},l9oz3z,False,False,False,False,False,True,False,,[],dark,text,False,NaN,NaN,False,True,0,0,False,all_ads,/r/teslainvestorsclub/comments/l9oz3z/my_respe...,False,NaN,NaN,6.0,1612420044,1,NaN,NaN,[removed],True,False,False,teslainvestorsclub,t5_n9evv,43882,public,confidence,default,NaN,NaN,"My respected TESLA investors, I journaled my u...",0,[],1.00,https://www.reddit.com/r/teslainvestorsclub/co...,NaN,all_ads,6.0,1.612141e+09,01df4ac6-4163-11ea-8c44-0ea4540c1e5b,Multi-Topic,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-01
2,[],False,[deleted],,None,NaN,NaN,None,dark,NaN,NaN,NaN,NaN,[],False,False,2021-02-01 05:51:04,apple.news,https://www.reddit.com/r/teslainvestorsclub/co...,{},l9mzj2,False,False,False,False,False,False,False,,[],dark,text,False,NaN,NaN,False,True,0,0,False,all_ads,/r/teslainvestorsclub/comments/l9mzj2/thoughts...,False,NaN,NaN,6.0,1612415647,1,NaN,NaN,[deleted],True,False,False,teslainvestorsclub,t5_n9evv,43865,public,confidence,default,93.0,140.0,Thoughts on this article on Tesla profitabilit...,0,[],1.00,https://apple.news/A-jG3gIIQQQO1S7HHrvxuOQ,https://apple.news/A-jG3gIIQQQO1S7HHrvxuOQ,all_ads,6.0,NaN,80519a3a-50db-11ea-959d-0e3217d46273,Energy,deleted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-01
3,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",False,swashbuckler-27,NaN,None,[],065f1cbc-ae10-11e8-86a9-0e9061152564,Text Only,dark,text,t2_d23bl,False,False,[],False,False,2021-02-01 05:33:03,youtu.be,https://www.reddit.com/r/teslainvestorsclub/co...,{'gid_1': 1},l9mlbj,True,False,False,False,True,False,False,,[],dark,text,False,{'oembed': {'au

In [36]:
# Clean Scraped Data from TIC Posts

TICP["created_utc"] = TICP["created_utc"].apply(lambda x: datetime.fromtimestamp(x)) #Used only for first-time runs

TICP["Date"] = pd.to_datetime(TICP["created_utc"],format=('%Y%m%d')).dt.date
variables = ["title","Date","created_utc","num_comments","permalink","subreddit"]
TICPosts= TICP[variables]
TICPosts["Post/Comment"] = "P"
TICPosts.head()

#TICPosts.to_csv("TICP.csv",header=True) ## Saving to csv format

C:\Users\herow\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,Date,created_utc,num_comments,permalink,subreddit,Post/Comment
0,184k 1Q21 &amp; 871k 2021 Delivery Estimate - ...,2021-02-01,2021-02-01 07:46:55,50,/r/teslainvestorsclub/comments/l9pe4y/184k_1q2...,teslainvestorsclub,P
1,"My respected TESLA investors, I journaled my u...",2021-02-01,2021-02-01 07:25:43,0,/r/teslainvestorsclub/comments/l9oz3z/my_respe...,teslainvestorsclub,P
2,Thoughts on this article on Tesla profitabilit...,2021-02-01,2021-02-01 05:51:04,0,/r/teslainvestorsclub/comments/l9mzj2/thoughts...,teslainvestorsclub,P
3,Is Tesla in a ‘retail bubble’? Q4 Earnings and...,2021-02-01,2021-02-01 05:33:03,2,/r/teslainvestorsclub/comments/l9mlbj/is_tesla...,teslainvestorsclub,P
4,Volkswagen Unveils 50-Foot Solar-Powered Catam...,2021-02-01,2021-02-01 04:20:33,2,/r/teslainvestorsclub/comments/l9kzx9/volkswag...,teslainvestorsclub,P


In [61]:
posts = pd.concat([WSBPosts,TICPosts])

In [63]:
posts.to_csv("Postslatest.csv",header=True)